In [1]:
import torch
import numpy as np
import os,sys
sys.path.append('../..')
#parametres 
data_set_name='gowalla'
device='cuda:0'
rerank_topk=20
topk=20
optimizer=lambda params: torch.optim.Adam(params, lr=1e-3, amsgrad=True)
have_processed_data=True
tree_num=1
model_num=2
num_layers=2#layer of transformer
k=18#the branch of each tree
n_head=4
d_model=96


train_sample_seg_cnt=10#the training data is located in the train_sample_seg_cnt datafiles
parall=10
seq_len=70 # se_len-1 is the number of behaviours in all the windows
min_seq_len=15
test_user_num=1000# the number of user in test file
raw_data_file='../../data/{}/{}.txt'.format(data_set_name,data_set_name)
train_instances_file='../../data/{}/train_instances'.format(data_set_name)
test_instances_file='../../data/{}/test_instances'.format(data_set_name)
validation_instances_file='../../data/{}/validation_instances'.format(data_set_name)
item_num_node_num_file='../../data/{}/item_node_num.txt'.format(data_set_name)
train_item_vec_file='../../data/{}/train_item_vec.npy'.format(data_set_name)
#item_to_code_file='../../data/{}/item_to_code.npy'.format(data_set_name)
#code_to_item_file='../../data/{}/code_to_item.npy'.format(data_set_name)
DIN_Model_path='../../data/{}/DIN_MODEL.pt'.format(data_set_name)
tree_has_generated=False
max_iters=100
reranker="Trm"
total_batch_num=20000

test_batch_size=100
assert tree_num==1

eps=0.000001
if device!='cpu':
    torch.cuda.set_device(device)
    device='cuda'
%load_ext autoreload
%autoreload 2
from lib import Trm4Rec

In [2]:
##load DIN model which is the ranker for fine-ranking
from lib import DeepInterestNetwork as DIN
DIN_Model=torch.load(DIN_Model_path,map_location=torch.device(device))
DIN_Model.eval()
pass

In [3]:
import lib
from lib import generate_train_and_test_data
import gc
import numpy as np
if not have_processed_data: 
    behavior_dict, train_sample, test_sample ,user_num,item_num,item_vec= generate_train_and_test_data._read(raw_data_file, 'train.dat', 'test.dat',
                                                                            test_user_num)  # 20 is the test users
    # write the training instance into different train_sample_seg_cnt files， avoid that a file is too large
    # stat record the click frequency of each item
    # seq_len=20 min that 19 behaviors and one label
    stat = generate_train_and_test_data._gen_train_sample(train_sample, train_instances_file,test_sample=test_sample,
                                                    train_sample_seg_cnt=train_sample_seg_cnt,
                                                    parall=parall, seq_len=seq_len, min_seq_len=min_seq_len)
    generate_train_and_test_data._gen_test_sample(test_sample, test_instances_file, seq_len=seq_len,
                                            min_seq_len=min_seq_len)
    del behavior_dict
    del train_sample
    del test_sample
    del stat
    np.savetxt(item_num_node_num_file,np.array([user_num,item_num]),fmt='%d',delimiter=',')
else:
    [user_num,item_num]=np.loadtxt(item_num_node_num_file,dtype=np.int32,delimiter=',')
print('user num is {}, item is {}'.format(user_num,item_num))

user num is 13583, item is 71436


In [ ]:
init_way_list=['embkm']
feature_ratio_list = [1.0]
for init_way,feature_ratio in zip(init_way_list,feature_ratio_list):  
    item_to_code_file_list=[]
    code_to_item_file_list=[]
    for tree_id in range(model_num):
        item_to_code_file=['../../data/{}/tree/{}{}_item_to_code_tree_id_{}_k{}.npy'.format(data_set_name,init_way,feature_ratio,tree_id+1,k)]
        code_to_item_file=['../../data/{}/tree/{}{}_code_to_item_tree_id_{}_k{}.npy'.format(data_set_name,init_way,feature_ratio,tree_id+1,k)]
        item_to_code_file_list.append(item_to_code_file)
        code_to_item_file_list.append(code_to_item_file)
    if not tree_has_generated:
        if init_way == 'embkm':
            data=DIN_Model.item_embedding.embed.weight.data[:item_num,:].cpu()
        elif init_way == 'datakm':
            data=torch.tensor(np.load(train_item_vec_file),dtype=torch.float32)
        else:
            data=None
    else:
        data=None
    train_model_list = []
    for i in range(model_num):
        train_model = Trm4Rec(item_num=int(item_num),
                              user_seq_len=seq_len-1,
                              d_model=d_model,
                              nhead=n_head,
                              tree_num=tree_num,
                              device=device,
                              optimizer=optimizer,
                              num_layers=num_layers,
                              k=k,
                              item_to_code_file_list=item_to_code_file_list[i],
                              code_to_item_file_list=code_to_item_file_list[i],
                              tree_has_generated=tree_has_generated,
                              init_way=init_way,
                              max_iters=max_iters,
                              feature_ratio=feature_ratio,
                              data=data,#used for kmeans tree
                              parall=parall,
                              ranker=None)
        train_model_list.append(train_model)

get into cluster training process
start to cluster
Train iteration done, pcode:0, data size: 1048576, elapsed time: 1137.5635221004486
Train iteration done, pcode:1, data size: 65536, elapsed time: 603.2157077789307
Train iteration done, pcode:2, data size: 65536, elapsed time: 627.9392492771149
Train iteration done, pcode:3, data size: 65536, elapsed time: 635.7078597545624
Train iteration done, pcode:4, data size: 65536, elapsed time: 635.0993638038635
Train iteration done, pcode:5, data size: 65536, elapsed time: 636.0720002651215
Train iteration done, pcode:6, data size: 65536, elapsed time: 634.3437657356262
Train iteration done, pcode:7, data size: 65536, elapsed time: 632.1353142261505
Train iteration done, pcode:8, data size: 65536, elapsed time: 634.7032041549683
Train iteration done, pcode:9, data size: 65536, elapsed time: 652.4354150295258
Train iteration done, pcode:10, data size: 65536, elapsed time: 642.8248453140259
Train iteration done, pcode:11, data size: 65536, elap

Train iteration done, pcode:100, data size: 4096, elapsed time: 207.191632270813
Train iteration done, pcode:101, data size: 4096, elapsed time: 207.22353076934814
Train iteration done, pcode:102, data size: 4096, elapsed time: 206.41566729545593
Train iteration done, pcode:103, data size: 4096, elapsed time: 206.48476910591125
Train iteration done, pcode:104, data size: 4096, elapsed time: 206.18183159828186
Train iteration done, pcode:105, data size: 4096, elapsed time: 208.4209816455841
Train iteration done, pcode:106, data size: 4096, elapsed time: 204.63563418388367
Train iteration done, pcode:107, data size: 4096, elapsed time: 209.00358986854553
Train iteration done, pcode:108, data size: 4096, elapsed time: 205.0094292163849
Train iteration done, pcode:109, data size: 4096, elapsed time: 209.6157670021057
Train iteration done, pcode:110, data size: 4096, elapsed time: 206.36543798446655
Train iteration done, pcode:111, data size: 4096, elapsed time: 208.42183637619019
Train ite

Train iteration done, pcode:200, data size: 4096, elapsed time: 229.96173644065857
Train iteration done, pcode:201, data size: 4096, elapsed time: 231.0992181301117
Train iteration done, pcode:202, data size: 4096, elapsed time: 228.70111227035522
Train iteration done, pcode:203, data size: 4096, elapsed time: 229.6554193496704
Train iteration done, pcode:204, data size: 4096, elapsed time: 228.5895299911499
Train iteration done, pcode:205, data size: 4096, elapsed time: 230.0982005596161
Train iteration done, pcode:206, data size: 4096, elapsed time: 231.44983959197998
Train iteration done, pcode:207, data size: 4096, elapsed time: 229.21680974960327
Train iteration done, pcode:208, data size: 4096, elapsed time: 227.7284152507782
Train iteration done, pcode:209, data size: 4096, elapsed time: 228.43782091140747
Train iteration done, pcode:210, data size: 4096, elapsed time: 231.4256854057312
Train iteration done, pcode:211, data size: 4096, elapsed time: 230.45215225219727
Train iter